### IMPORTING LIBRARIES

In [6]:
# Importing standard python libraies
import time
import sys


# Importing third-party libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tqdm


***

### DECLARING VARIABLES

In [52]:
# Url list and variables
steam_url = 'https://steamspy.com/'

page_year = {
    '2008' : 'https://steamspy.com/year/2008',
    '2009' : 'https://steamspy.com/year/2009',
    '2010' : 'https://steamspy.com/year/2010',
    '2011' : 'https://steamspy.com/year/2011',
    '2012' : 'https://steamspy.com/year/2012',
    '2013' : 'https://steamspy.com/year/2013',
    '2014' : 'https://steamspy.com/year/2014',
    '2015' : 'https://steamspy.com/year/2015',
    '2016' : 'https://steamspy.com/year/2016',
    '2017' : 'https://steamspy.com/year/2017',
    '2018' : 'https://steamspy.com/year/2018',
    '2019' : 'https://steamspy.com/year/2019',
    '2020' : 'https://steamspy.com/year/2020',
    '2021' : 'https://steamspy.com/year/2021'
}

game_data_dict = {
    'id':[], 
    'name':[], 
    'dev':[], 
    'publisher':[], 
    'genre':[], 
    'language':[], 
    'tags':[], 
    'category':[], 
    'release':[], 
    'actual_price':[], 
    'discount':[],
    'old_userscore':[],
    'metascore':[], 
    'owners':[],
    'followers':[]
    }


found_dev= []
found_genre = []
found_publisher = []
found_genre = []
found_language = []
found_tags = []
found_category = []
found_release_date = []
found_price = []
found_discount = []
found_old_userscore = []
found_metascore = []
found_owners = []
found_followers = []

***

### CREATING FUNCTIONS

In [53]:
# request sending and error handling

# -------------------------------->Takes one and only one url from page_year dict as argument
def get_soup(url: str) -> str:
    response = requests.get(url)
# -------------------------------->Parse the html in case of positive response
    try:
        if response.status_code == 200:
            soup = BeautifulSoup(response.text,"html.parser")
            #print('HTTP 200 OK -- SUCCESSFUL REQUEST')
            return soup
# -------------------------------->Exception handling needs to be improved (under development).
    except:
        print('error')


# -------------------------------->Takes get_soup function as argument
def get_game_apps(soup):
    games_links = soup.find_all("tr")
    games_links.pop(0)


    for game in games_links:
        game_data_dict['id'].append(game.a["href"])
        game_data_dict['name'].append(game.contents[3].text.strip())


# -------------------------------->Filters the section of the html that contains the data of a set in the name list from game_data_dict
def get_game_tags_info(game_app):
    game_url = steam_url+game_app
    response = requests.get(game_url)
    try:
        if response.status_code == 200:
            soup = BeautifulSoup(response.text,"html.parser")
# -------------------------------->Extract the game info
            #print('HTTP 200 OK -- SUCCESSFUL REQUEST FOR GAME APP')
            try:
                game_soup = soup.find("div",attrs={"class":"col-md-4 no-padding"}).find("p")
                return game_soup
            except:
                game_data_dict['dev'].append(None)
                game_data_dict['publisher'].append(None)
                game_data_dict['genre'].append(None)
                game_data_dict['language'].append(None)
                game_data_dict['tags'].append(None)
                game_data_dict['category'].append(None)
                game_data_dict['release'].append(None)
                game_data_dict['actual_price'].append(None)
                game_data_dict['discount'].append(None)
                game_data_dict['old_userscore'].append(None)
                game_data_dict['metascore'].append(None)
                game_data_dict['owners'].append(None)
                game_data_dict['followers'].append(None)
                return 'FORMAT ERROR'
    except:
        print('error')

# --------------------------------> Iterates in each app id to get the tags info

def do_all():
    global found_dev
    global found_genre
    global found_publisher
    global found_genre
    global found_language
    global found_tags
    global found_category
    global found_release_date
    global found_price
    global found_discount
    global found_old_userscore
    global found_metascore
    global found_owners
    global found_followers

    for i in tqdm.tqdm(range(0, len(game_data_dict['id']))):
        
        if get_game_tags_info(game_data_dict['id'][i]) == 'FORMAT ERROR':
            continue
        else:
            info = get_game_tags_info(game_data_dict['id'][i])

    # --------------------------------> extract data from every game
            for ahref in info.find_all('a'): # for ahref
                if 'dev' in str(ahref):
                    found_dev.append(ahref.text)
                
                elif 'genre' in str(ahref):
                    found_genre.append(ahref.text)

                elif 'language' in str(ahref):
                    found_language.append(ahref.text)
                
                elif 'tag' in str(ahref):
                    found_tags.append(ahref.text)

            for strong in info.find_all('strong'): # for strong
                if 'Publisher' in str(strong):
                    found_publisher.append(strong.nextSibling.nextSibling.text)

                elif 'Category' in str(strong):
                    found_category.extend(strong.nextSibling.strip().split(', '))

                elif 'Release date' in str(strong):
                    found_release_date.append(strong.nextSibling.replace(':','').replace('(previously','').strip())

                elif 'Price' in str(strong):
                    found_price.append(strong.nextSibling.strip())

                elif 'Old userscore' in str(strong):
                    found_old_userscore.append(strong.nextSibling.strip())

                elif 'Metascore' in str(strong):
                    found_metascore.append(strong.nextSibling.strip())

                elif 'Owners' in str(strong):
                    found_owners.append(strong.nextSibling.replace(r': ','').strip().replace(u'\xa0..\xa0',u' - '))

                elif 'Followers' in str(strong):
                    found_followers.append(strong.nextSibling.replace(': ',''))

            for font in info.find_all('font'):
                found_discount.append(font.text)

            

            game_data_dict['dev'].append(list(set(found_dev)))
            game_data_dict['publisher'].append(found_publisher)
            game_data_dict['genre'].append(found_genre)
            game_data_dict['language'].append(found_language)
            game_data_dict['tags'].append(found_tags)
            game_data_dict['category'].append(found_category)
            game_data_dict['release'].append(found_release_date)
            game_data_dict['actual_price'].append(found_price)
            game_data_dict['discount'].append(found_discount)
            game_data_dict['old_userscore'].append(found_old_userscore)
            game_data_dict['metascore'].append(found_metascore)
            game_data_dict['owners'].append(found_owners)
            game_data_dict['followers'].append(found_followers)


            found_dev= []
            found_genre = []
            found_publisher = []
            found_genre = []
            found_language = []
            found_tags = []
            found_category = []
            found_release_date = []
            found_price = []
            found_discount = []
            found_old_userscore = []
            found_metascore = []
            found_owners = []
            found_followers = []

***

### GETTING ID AND NAME DATA

In [54]:
# Select the year in this cell, then run
get_soup(page_year['2021'])
get_game_apps(get_soup(page_year['2021']))

***

### GETTING THE REST OF THE DATA, BE PATIENT!

In [ ]:
do_all()

***

### DATAFRAME CREATION

In [ ]:
df = pd.DataFrame(game_data_dict)